streamlit, chainlit, gradio 세개의 ui 툴 예제

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
%xterm

# 터미널 내에 입력 (두줄 따로 입력)
"""
curl -fsSL https://ollama.com/install.sh | sh 
ollama serve & ollama pull llama3
"""

In [ ]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3")
llm.invoke("Tell me 3 red flower names")

In [ ]:
!pip install -q streamlit
!pip install streamlit_chat
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install chromadb
!pip install pypdf
!pip install fastembed

In [ ]:
%%writefile rag.py

from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain.vectorstores.utils import filter_complex_metadata

class ChatPDF:
    vector_store = None
    retriever = None
    chain = None
    
    def __init__(self):
        self.model = ChatOllama(model="llama3")
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
        self.prompt = PromptTemplate.from_template(
            """
            <s> [INST] you are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
            If you don't konw the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. [/INST] </s>
            [INST] Question: {question}
            Context: {context}
            Answer: [/INST]
            """
        )

    def ingest(self, pdf_file_path: str):
        docs = PyPDFLoader(file_path=pdf_file_path).load()
        chunks = self.text_splitter.split_documents(docs)
        chunks = filter_complex_metadata(chunks)

        vector_store = Chroma.from_documents(documents=chunks, embedding=FastEmbedEmbeddings())
        self.retriever = vector_store.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={
                "k": 3,
                "score_threshold": 0.5
            },
        )

        self.chain = (
            {"context": self.retriever, "question": RunnablePassthrough()}
            | self.prompt
            | self.model
            | StrOutputParser()
        
        )

    def ask(self, query: str):
        if not self.chain:
            return "Please, add a PDF document first."
        
        return self.chain.invoke(query)

    def clear(self):
        self.vector_store = None
        self.retriever = None
        self.chain = None


        


In [ ]:
%%writefile app.py
from lang
import os
import tempfile
import streamlit as st
from streamlit_chat import message
from rag import ChatPDF

st.set_page_config(page_title="ChatPDF")

def display_messages():
    st.subheader("Chat")
    for i, (msg, is_user) in enumerate(st.session_state["messages"]):
        message(msg, is_user=is_user, key=str(i))

    st.session_state["thinking_spinner"] = st.empty()

def process_input():
    if st.session_state["user_input"] and len(st.session_state["user_input"].strip()) > 0:
        user_text= st.session_state["user_input"].strip()
        with st.session_state["thinking_spinner"], st.spinner(f"Thinking"):
            agent_text = st.session_state["assistant"].ask(user_text)

        st.session_state["messages"].append((user_text, True))
        st.session_state["messages"].append((agent_text, False))

# pdf 업로드시 저장
def read_and_save_file():
    st.session_state["assistant"].clear()
    st.session_state["messages"] = []
    st.session_state["user_input"] = ""

    for file in st.session_state["file_uploader"]:
        with tempfile.NamedTemporaryFile(delete=False) as tf:
            tf.write(file.getbuffer())
            file_path = tf.name

        with st.session_state["ingestion_spinner"], st.spinner(f"Ingesting {file.name}"):
            st.session_state["assistant"].ingest(file_path)
        os.remove(file_path)


def page():
    if len(st.session_state) == 0:
        st.session_state["messages"] = []
        st.session_state["assistant"] = ChatPDF()


    st.header("ChatPDF")

    st.subheader("Upload a document")
    st.file_uploader(
        "Upload document",
        type=["pdf"],
        key="file_uploader",
        on_change=read_and_save_file,
        label_visibility="collapsed",
        accept_multiple_files=True,
    )

    st.session_state["ingestion_spinner"] = st.empty()

    display_messages()
    st.text_input("Message", key="user_input", on_change=process_input)

    
if __name__ == "__main__":
    page()
    print("a")

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader

DATA_PATH="/content/drive/MyDrive/gdrive/data/test"
DB_PATH="./vectorstores/db/"

def create_vector_db():
    print(DATA_PATH)
    loader = PyPDFDirectoryLoader(DATA_PATH)
    documents = loader.load()
    print(f"Processed {len(documents)} pdf files")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    vectorstore = Chroma.from_documents(documents=texts, embedding=FastEmbedEmbeddings(), persist_directory=DB_PATH)
    vectorstore.persist()

if __name__ == "__main__":
    create_vector_db()

In [ ]:
!pip install langchain chainlit pyngrok openai google-search-results

In [ ]:
%%writefile chainlit_app.py

from langchain import hub
from langchain_community.embeddings import GPT4AllEmbeddings, FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.callbacks.manager import Callbackmanager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import chainlit as cl

QA_CHAIN_PROMPT = hub.pull('rlm/rag-prompt-mistral')

def load_llm():
    llm = Ollama(
        model="llama3",
        verbose=True,
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
    )
    return llm

def retrieval_qa_chain(llm, vectorstore):
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True
    )
    return qa_chain

def qa_bot():
    llm = load_llm()
    DB_PATH="vectorstores/db/"
    vectorstore=Chroma(persist_directory=DB_PATH, embedding_function=FastEmbedEmbeddings())
    qa = retrieval_qa_chain(llm, vectorstore)
    return qa

@cl.on_chat_start
async def start():
    chain=qa_bot()
    msg=cl.Message(content="firing up the research info bot...")
    await msg.sned()
    msg.content = "Hi, welcom to the research info bot. What is your query?"
    await msg.update()
    cl.useer_session.set("chain", chain)


@cl.on_message
async def main(message):
    chain = cl.user_session.get("chain")
    cb = cl.AsyncLangchainCallbackHandler(
        stream_final_answer=True,
        answer_prefix_tokens=["FINAL", "ANSWER"]
    )
    cb.answer_reached=True

    res = await chain.acall(message.content, callbacks=[cb])
    print(f"response: {res}")
    answer = res["result"]
    answer = answer.replace(".", ".\n")
    sources = res["source_documents"]

    if sources:
        answwer += f"\nSources: " + str(str(sources))
    else:
        answer += f"\nNo Sources found"

    await cl.Message(content=answer).send()


In [ ]:
!chainlit run chainlit_app.py &>/content/logs.txt &

In [ ]:
!ngrok config add-authtoken [token]

In [ ]:
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
%xterm
# 터미널 내에 입력
"""
curl -fsSL https://ollama.com/install.sh | sh
ollama serve & ollama pull llama3
ollama pull nomic-embed-text
"""

In [ ]:
!pip install langchain 
!pip install langchain-core
!pip install langchain-community

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3")
llm.invoke("What is the Mesaning of life")

In [ ]:
!pip install ollama langchain beautifulsoup4 chromadb gradio -q

In [ ]:
import gradio as gr
import ollama
from bs4 import BeautifulSoup as bs
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Load the data from web URL
url = "https://en.wikipedia.org/wiki/Ohiya"
loader = WebBaseLoader(url)
docs = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Define the function to call the Ollama Llama3 model
def ollama_llm(questions, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3', messages={'role': 'user', 'content': formatted_prompt})
    return response['message']['content']

# RAG 셋업
retriever = vectorstore.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Gradio 인터페이스
def get_important_facts(question):
    return rag_chain(question)

# Gradio 앱 인터페이스 생성
iface = gr.Interface(
    fn=get_important_facts,
    inputs=gr.Textbox(lines=2, placeholder="enter your question here..."),
    outputs="text",
    title="RAG with Llama3",
    description="Ask questions about the proveded context"
)

iface.launch()

